In [ ]:
import pandas as pd
import seaborn as sns
import h5py
import numpy as np

# locs_path = '/home/miguel/Projects/smlm_z/autofocus/VIT_zeiss_lowsnr_data/out_resnet_fov-max_aug11/out_nup/locs_3d.hdf5'
# locs_path = '/home/miguel/Projects/smlm_z/autofocus/VIT_zeiss_lowsnr_data/out_33/out_nup/locs_3d.hdf5'
locs_path = '/home/miguel/Projects/smlm_z/autofocus/VIT_zeiss_lowsnr_data/out_24_nvidia6/out_nup/locs_3d.hdf5'
# locs_path = '/home/miguel/Projects/smlm_z/publication/VIT_fd-loco3/out_3/out_nup/locs_3d.hdf5'
df = pd.read_hdf(locs_path, key='locs')



In [ ]:
list(df)

In [ ]:
sns.scatterplot(data=df, x='x [nm]', y='y [nm]', hue='group')

In [ ]:
sns.scatterplot(data=df.groupby('frame').mean(), x='frame', y='z [nm]', alpha=0.05, label='z locs (nm)')

In [ ]:
from scipy.interpolate import splrep, BSpline
import matplotlib.pyplot as plt
rcc = 500
def frame_center(x):
    return x.left + (rcc/2)
    
df['frame_bin'] = (pd.cut(df['frame'].astype(int), np.arange(0, df['frame'].max()+rcc, rcc), include_lowest=True))
df_group_bin = df.groupby('frame_bin').mean().reset_index()
df_group_bin['frame_bin_center'] = df_group_bin['frame_bin'].map(frame_center)

sns.scatterplot(data=df_group_bin, x='frame_bin_center', y='z [nm]', label='Mean z pos (nm)')
from scipy.interpolate import CubicSpline

x = df_group_bin['frame_bin_center']
y = df_group_bin['z [nm]']
spline = BSpline(*splrep(x, y, s=len(y)*100))

_x = np.arange(df['frame'].min(), df['frame'].max()+1)
_y = spline(_x)
sns.lineplot(x=_x, y=_y, label='Spline fit')
plt.show()

In [ ]:
df2 = df.copy(deep=True)
del df2['frame_bin']
del df2['index']
df2['z [nm]'] -= df2['frame'].map(lambda f: _y[f])
sns.scatterplot(data=df2.groupby('frame').mean(), x='frame', y='z [nm]', alpha=0.02, label='raw drift')


In [ ]:
sns.histplot(data=df, x='z [nm]')
plt.show()
sns.histplot(data=df2, x='z [nm]')
plt.show()

In [ ]:
df2.shape

In [ ]:
import h5py
import shutil
locs_undrift_path = locs_path.replace('locs_3d.hdf5', f'locs_3d_undrift_z_rcc{rcc}.hdf5')

with h5py.File(locs_undrift_path, "w") as locs_file:
    locs_file.create_dataset("locs", data=df2.to_records())
shutil.copyfile(locs_path.replace('.hdf5', '.yaml'), locs_undrift_path.replace('.hdf5', '.yaml'))